In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
import plotly.offline as py
import plotly.graph_objs as go
from matplotlib import pyplot
import scipy.signal
plt.rcParams['figure.figsize'] = 16, 16
import re
import torch
from torch import nn, autograd
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import math
%matplotlib inline
data = pd.read_json("./Data/train.json")
test = pd.read_json("./Data/test.json")

def sampleImg(train,x):
    #returns a random sample set of 4 icebergs and ships
    icebergs = train[train.is_iceberg==1].sample(n=x)
    ships = train[train.is_iceberg==0].sample(n=x)
    
    return(ships,icebergs)
    
def common_size(a1, a2):
    #sets the size of two
    (r1, c1) = a1.shape
    (r2, c2) = a2.shape
    return (a1[r1-r2 if r1>r2 else 0:,
               c1-c2 if c1>c2 else 0:],
            a2[r2-r1 if r2>r1 else 0::,
               c2-c1 if c2>c1 else 0:])

def gradient(im):
    # Applies a gradient by using 2 simple 2d convolutions
    imv, imh = common_size(scipy.signal.convolve2d(im, [[1., -1.]],mode='valid',boundary='fill',fillvalue=0),
                           scipy.signal.convolve2d(im, [[1.], [-1.]],mode='valid',boundary='fill',fillvalue=0))
    return np.sqrt(np.power(imv, 2)+np.power(imh, 2))

#Edge detection kernel
def gradientEdgeDetect(im):
    imv, imh = common_size(scipy.signal.convolve2d(im, [[-1,0, -1],[-2,0,2],[-1,0,1]],mode='valid',boundary='fill',fillvalue=0),
                           scipy.signal.convolve2d(im, [[1,2,1], [0,0,0],[-1,-2,-1]],mode='valid',boundary='fill',fillvalue=0))
    return np.sqrt(np.power(imv, 2)+np.power(imh, 2))

#Edge detection kernel
def gradientEdgeDetect2(im):
    imv, imh = common_size(scipy.signal.convolve2d(im, [[-1,0, 1],[-1,0,1],[-1,0,1]],mode='valid',boundary='fill',fillvalue=0),
                           scipy.signal.convolve2d(im, [[1,1,1], [0,0,0],[-1,-1,-1]],mode='valid',boundary='fill',fillvalue=0))
    return np.sqrt(np.power(imv, 2)+np.power(imh, 2))

#Edge detection kernel
def gradientSobel(im):
    imv, imh = common_size(scipy.signal.convolve2d(im, [[1,0, -1],[2,0,-2],[1,0,-1]],mode='valid',boundary='fill',fillvalue=0),
                           scipy.signal.convolve2d(im, [[1,2,1], [0,0,0],[-1,-2,-1]],mode='valid',boundary='fill',fillvalue=0))
    return np.sqrt(np.power(imv, 2)+np.power(imh, 2))

def gradientLaplace(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,-1, 0],[-1,4,-1],[0,-1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def gradientLaplace2(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,-1, 0],[-1,8,-1],[0,-1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def gradientLaplace3(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,-1, 0],[-1,5,-1],[0,-1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def gradientLaplace4(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,1, 0],[1,-4,1],[0,1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def gradientLaplace5(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,1, 0],[1,-5,1],[0,1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def gradientLaplace6(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,1, 0],[1,8,1],[0,1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def gradientLaplace7(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,1, 0],[1,9,1],[0,1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def gradientLaplace8(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,1, 0],[1,-8,1],[0,1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def gradientLaplace9(im):
    #Smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,1, 0],[1,-9,1],[0,1,0]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def bigLaplace1(im):
    #Large smoothing gradient
    imh = scipy.signal.convolve2d(im, [[0,0,0,0,0],[0,0,-1,0,0],[0,-1,8,-1,0],[0,0,-1,0,0],[0,0,0,0,0]],
                                  mode='valid',boundary='fill',fillvalue=0)
    return imh

def Emboss(im):
    # Convolution that adds an emboss effect to a 2d array
    imh = scipy.signal.convolve2d(im, [[-2,-1, 0],[-1,1,1],[0,1,2]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def SobelBottom(im):
    # Sobel convoltion that detects bottom edges
    imh = scipy.signal.convolve2d(im, [[-1,-2, -1],[0,0,0],[1,2,1]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def SobelTop(im):
    # Sobel convoltion that detects top edges
    imh = scipy.signal.convolve2d(im, [[1,2, 1],[0,0,0],[-1,-2,-1]],mode='valid',boundary='fill',fillvalue=0)
    return imh
    
def SobelLeft(im):
    # Sobel convoltion that detects left edges
    imh = scipy.signal.convolve2d(im, [[1,0, -1],[2,0,-2],[1,0,-1]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def SobelRight(im):
    # Sobel convolution that detects right edges
    imh = scipy.signal.convolve2d(im, [[-1,0, 1],[-2,0,2],[-1,0,1]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def outline(im):
    #Edge detection convolution
    imh = scipy.signal.convolve2d(im, [[-1,-1, -1],[-1,8,-1],[-1,-1,-1]],mode='valid',boundary='fill',fillvalue=0)
    return imh

def multiProcess(list,i):
    # Takes a list of convolutions and applies them to an array
    transforms = list
    
    arr = np.reshape(np.array(data.iloc[i,0]),(75,75))
    
    for j in transforms:
        arr=skimage.transform.resize(j(arr),(75,75))
             
    return arr

def callSet(list,dataset):
    # Iterates through the data set and perfoms a list of changes for preprocessing
    i=0
    for index, row in dataset.iterrows():
        dataset['band_1'][i] = multiProcess(list,i)
        dataset['band_2'][i] = multiProcess(list,i)
        i=i+1
    return dataset

def multiProcess2(list,i):
    # Takes a list of convolutions and applies them to an array
    transforms = list
    
    arr = np.reshape(np.array(test.iloc[i,0]),(75,75))
    
    for j in transforms:
        arr=skimage.transform.resize(j(arr),(75,75))
             
    return arr

def callSet2(list,dataset):
    # Iterates through the data set and perfoms a list of changes for preprocessing
    i=0
    for index, row in dataset.iterrows():
        dataset['band_1'][i] = multiProcess(list,i)
        dataset['band_2'][i] = multiProcess(list,i)
        i=i+1
    return dataset

def addFlip(dataset):
    # Flips 50% of the pictures and adds to the data passed
    original = dataset
    sub = dataset.sample(frac=0.5)
    dataset = dataset[~dataset.isin(sub)].dropna()
    for index, row in dataset.iterrows():
        dataset['band_1'] = np.flipud(dataset['band_1'])
        dataset['band_2'] = np.flipud(dataset['band_2'])
    
    original = original.append(pd.DataFrame(data = dataset), ignore_index=True)
    
    return original

def fillHoles(dataset):
    # Applies a filter for filling holes
    for index, row in dataset.iterrows():
        
        img = np.reshape((dataset['band_1'][index]),(75,75))
        img = rescale_intensity(img)
        seed = np.copy(img)
        seed[1:-1, 1:-1] = img.max()
        mask = img
        filled = reconstruction(seed, mask, method='erosion')
        filled = skimage.transform.resize(filled,(75,75))
        dataset['band_1'][index] = filled

        img = np.reshape(np.array(dataset['band_2'][index]),(75,75))
        img = rescale_intensity(img)
        seed = np.copy(img)
        seed[1:-1, 1:-1] = img.max()
        mask = img
        filled = reconstruction(seed, mask, method='erosion')
        filled = skimage.transform.resize(filled,(75,75))
        dataset['band_2'][index] = filled
        
        print(index,' of ',len(dataset))
    
    return dataset
   

In [41]:
from skimage import filters
from skimage import data as skd
from skimage.morphology import reconstruction
import warnings
warnings.filterwarnings('ignore')

# This segment takes lists of gradients to pass over our data and creates new data sests
#  It has a step for the training data and the validation set
#  On the training data, we also add 50% of the records to our set, but flip the images

list1=[gradientLaplace2,gradientLaplace6,gradientLaplace7,
      gradientLaplace7,gradientEdgeDetect2,gradientLaplace7
        ,gradientLaplace9,gradientLaplace2
      ]
list2=[gradientLaplace2,gradientLaplace6,gradientLaplace7,
      gradientLaplace7,gradientEdgeDetect2,gradientLaplace7
        ,gradientLaplace9,gradientLaplace2,gradientEdgeDetect2,gradientLaplace9
      ]
list3=[gradientLaplace2,gradientLaplace6,gradientLaplace7,
      gradientLaplace7,gradientEdgeDetect2,gradientLaplace7
        ,gradientLaplace9,gradientLaplace2,gradientEdgeDetect2,gradientLaplace9,Emboss
      ]
list4=[gradientLaplace2,gradientLaplace6,gradientLaplace7,
      gradientLaplace7,gradientEdgeDetect2,gradientLaplace7
        ,gradientLaplace9,gradientLaplace2,Emboss
      ]
data1 = callSet(list1,data)
data1 = addFlip(data1)
data1.to_pickle('AugmententData1.pk1')
data2 = callSet(list2,data)
data2 = addFlip(data2)
data2.to_pickle('AugmententData2.pk1')
data3 = callSet(list3,data)
data3 = addFlip(data3)
data3.to_pickle('AugmententData3.pk1')
data4 = callSet(list4,data)
data4 = addFlip(data4)
data4.to_pickle('AugmententData4.pk1')
data5 = fillHoles(data)
data5 = addFlip(data5)
data5.to_pickle('AugmententData5.pk1')



0  of  1604
1  of  1604
2  of  1604
3  of  1604
4  of  1604
5  of  1604
6  of  1604
7  of  1604
8  of  1604
9  of  1604
10  of  1604
11  of  1604
12  of  1604
13  of  1604
14  of  1604
15  of  1604
16  of  1604
17  of  1604
18  of  1604
19  of  1604
20  of  1604
21  of  1604
22  of  1604
23  of  1604
24  of  1604
25  of  1604
26  of  1604
27  of  1604
28  of  1604
29  of  1604
30  of  1604
31  of  1604
32  of  1604
33  of  1604
34  of  1604
35  of  1604
36  of  1604
37  of  1604
38  of  1604
39  of  1604
40  of  1604
41  of  1604
42  of  1604
43  of  1604
44  of  1604
45  of  1604
46  of  1604
47  of  1604
48  of  1604
49  of  1604
50  of  1604
51  of  1604
52  of  1604
53  of  1604
54  of  1604
55  of  1604
56  of  1604
57  of  1604
58  of  1604
59  of  1604
60  of  1604
61  of  1604
62  of  1604
63  of  1604
64  of  1604
65  of  1604
66  of  1604
67  of  1604
68  of  1604
69  of  1604
70  of  1604
71  of  1604
72  of  1604
73  of  1604
74  of  1604
75  of  1604
76  of  1604
77  of  1

594  of  1604
595  of  1604
596  of  1604
597  of  1604
598  of  1604
599  of  1604
600  of  1604
601  of  1604
602  of  1604
603  of  1604
604  of  1604
605  of  1604
606  of  1604
607  of  1604
608  of  1604
609  of  1604
610  of  1604
611  of  1604
612  of  1604
613  of  1604
614  of  1604
615  of  1604
616  of  1604
617  of  1604
618  of  1604
619  of  1604
620  of  1604
621  of  1604
622  of  1604
623  of  1604
624  of  1604
625  of  1604
626  of  1604
627  of  1604
628  of  1604
629  of  1604
630  of  1604
631  of  1604
632  of  1604
633  of  1604
634  of  1604
635  of  1604
636  of  1604
637  of  1604
638  of  1604
639  of  1604
640  of  1604
641  of  1604
642  of  1604
643  of  1604
644  of  1604
645  of  1604
646  of  1604
647  of  1604
648  of  1604
649  of  1604
650  of  1604
651  of  1604
652  of  1604
653  of  1604
654  of  1604
655  of  1604
656  of  1604
657  of  1604
658  of  1604
659  of  1604
660  of  1604
661  of  1604
662  of  1604
663  of  1604
664  of  1604
665  o

1168  of  1604
1169  of  1604
1170  of  1604
1171  of  1604
1172  of  1604
1173  of  1604
1174  of  1604
1175  of  1604
1176  of  1604
1177  of  1604
1178  of  1604
1179  of  1604
1180  of  1604
1181  of  1604
1182  of  1604
1183  of  1604
1184  of  1604
1185  of  1604
1186  of  1604
1187  of  1604
1188  of  1604
1189  of  1604
1190  of  1604
1191  of  1604
1192  of  1604
1193  of  1604
1194  of  1604
1195  of  1604
1196  of  1604
1197  of  1604
1198  of  1604
1199  of  1604
1200  of  1604
1201  of  1604
1202  of  1604
1203  of  1604
1204  of  1604
1205  of  1604
1206  of  1604
1207  of  1604
1208  of  1604
1209  of  1604
1210  of  1604
1211  of  1604
1212  of  1604
1213  of  1604
1214  of  1604
1215  of  1604
1216  of  1604
1217  of  1604
1218  of  1604
1219  of  1604
1220  of  1604
1221  of  1604
1222  of  1604
1223  of  1604
1224  of  1604
1225  of  1604
1226  of  1604
1227  of  1604
1228  of  1604
1229  of  1604
1230  of  1604
1231  of  1604
1232  of  1604
1233  of  1604
1234  of  

IndexError: single positional indexer is out-of-bounds

In [ ]:
test1 = callSet2(list1,test)
test1.to_pickle('AugmententTest1.pk1')
test2 = callSet2(list2,test)
test2.to_pickle('AugmententTest2.pk1')
test3 = callSet2(list3,test)
test3.to_pickle('AugmententTest3.pk1')
test4 = callSet2(list4,test)
test4.to_pickle('AugmententTest4.pk1')
test5 = fillHoles(test)
test5.to_pickle('AugmententTest5.pk1')

In [9]:

### Set data and validation 

data = data
test = test

### Prepare the data for nn

data['band_1'] = data['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
data['band_2'] = data['band_2'].apply(lambda x: np.array(x).reshape(75, 75))
data['inc_angle'] = pd.to_numeric(data['inc_angle'], errors='coerce')
test['band_1'] = test['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
test['band_2'] = test['band_2'].apply(lambda x: np.array(x).reshape(75, 75))
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
train = data.sample(frac=0.75)
val = data[~data.isin(train)].dropna()
band_1_tr = np.concatenate([im for im in train['band_1']]).reshape(-1, 75, 75)
band_2_tr = np.concatenate([im for im in train['band_2']]).reshape(-1, 75, 75)
full_tr = np.stack([band_1_tr, band_2_tr], axis=1)
band_1_val = np.concatenate([im for im in val['band_1']]).reshape(-1, 75, 75)
band_2_val = np.concatenate([im for im in val['band_2']]).reshape(-1, 75, 75)
full_val = np.stack([band_1_val, band_2_val], axis=1)
band_1_test = np.concatenate([im for im in test['band_1']]).reshape(-1, 75, 75)
band_2_test = np.concatenate([im for im in test['band_2']]).reshape(-1, 75, 75)
train_bands = torch.from_numpy(full_tr).float()
train_labels = torch.from_numpy(train['is_iceberg'].values).long()
train_set = TensorDataset(train_bands, train_labels)
val_bands = torch.from_numpy(full_val).float()
val_labels = torch.from_numpy(val['is_iceberg'].values).long()
val_set = TensorDataset(val_bands, val_labels)

def resample():
    train = data.sample(frac=0.75)
    val = data[~data.isin(train)].dropna()
    band_1_tr = np.concatenate([im for im in train['band_1']]).reshape(-1, 75, 75)
    band_2_tr = np.concatenate([im for im in train['band_2']]).reshape(-1, 75, 75)
    full_tr = np.stack([band_1_tr, band_2_tr], axis=1)
    band_1_val = np.concatenate([im for im in val['band_1']]).reshape(-1, 75, 75)
    band_2_val = np.concatenate([im for im in val['band_2']]).reshape(-1, 75, 75)
    full_val = np.stack([band_1_val, band_2_val], axis=1)
    band_1_test = np.concatenate([im for im in test['band_1']]).reshape(-1, 75, 75)
    band_2_test = np.concatenate([im for im in test['band_2']]).reshape(-1, 75, 75)
    train_bands = torch.from_numpy(full_tr).float()
    train_labels = torch.from_numpy(train['is_iceberg'].values).long()
    train_set = TensorDataset(train_bands, train_labels)
    val_bands = torch.from_numpy(full_val).float()
    val_labels = torch.from_numpy(val['is_iceberg'].values).long()
    val_set = TensorDataset(val_bands, val_labels)
    return train_set,val_set

train_set,val_set = resample()
full_test = np.stack([band_1_test, band_2_test], axis=1)
test_bands  = torch.from_numpy(full_test).float()

In [90]:
# Build nn

#Relu Decision - 
#https://arxiv.org/pdf/1505.00853.pdf

# Why Max Pool - 
#https://www.quora.com/What-is-the-benefit-of-using-average-pooling-rather-than-max-pooling

#Why Sigmoid -
#https://stats.stackexchange.com/questions/233658/softmax-vs-sigmoid-function-in-logistic-classifier

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.RRpool1 = torch.nn.Sequential(
            torch.nn.Conv2d(2, 32, kernel_size=(3,3),padding=2),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(.1),
            torch.nn.MaxPool2d((3,3),stride=(2,2))
        )
        
        self.RRpool2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=(3,3),padding=2),
            torch.nn.ReLU(),
            torch.nn.Dropout(.2),
            torch.nn.MaxPool2d((2,2),stride=(2,2))
        )
        
        self.RRpool3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3,padding=2),
            torch.nn.ReLU(),
            torch.nn.Dropout(.3),
            torch.nn.MaxPool2d((2,2),stride=(2,2))
        )
        
        self.res = torch.nn.Sequential(
            nn.Dropout(.1),
            torch.nn.Conv2d(128, 128, kernel_size=2,padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2),stride=(2,2))
        )
        
        self.classifier = torch.nn.Linear(6272, 1)             
    
        self.sig = nn.Sigmoid()
                   
    def forward(self, x):
        x = self.RRpool1(x)
        x = self.RRpool2(x)
        x = self.RRpool3(x)
        x = self.res(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)                
        x = self.sig(x)
        return x  

model = Net()
model = model.cuda()
print(model)

Net (
  (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (RRpool1): Sequential (
    (0): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): LeakyReLU (0.01)
    (2): Dropout (p = 0.1)
    (3): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (RRpool2): Sequential (
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU ()
    (2): Dropout (p = 0.2)
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (RRpool3): Sequential (
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU ()
    (2): Dropout (p = 0.3)
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (res): Sequential (
    (0): Dropout (p = 0.1)
    (1): Conv2d(128, 128, kernel_size=(2, 2), stride=(1, 1), padding=(2, 2))
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Linear (6272 -> 1)
  (sig): Sigmoid ()
)


In [95]:
loss_func = torch.nn.BCELoss()
criterion = loss_func
all_losses = []
val_losses = []
# What is adam optimizer?
#https://arxiv.org/pdf/1412.6980.pdf
optimizer = torch.optim.Adam(model.parameters(),lr=.0005,weight_decay=5e-5)

def fit (train, val, num_epochs, batch_size):
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_acc = 0.0
        model.train()
        train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val, batch_size=batch_size, shuffle=False)
        numerator = 0
        for data,target in train_loader:
            data, target = Variable(data.cuda(async=True)), Variable(target.cuda(async=True))
            numerator = numerator+1
            output = model(data.cuda())
            loss = criterion(output, target.float())
            running_loss += loss.data[0] * target.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            all_losses.append(running_loss / (batch_size * numerator))
            print('[{}/{}] Loss: {:.6f}'.format(
                epoch + 1, num_epochs, running_loss / (batch_size * numerator),
                running_acc / (batch_size * numerator)))
            print(running_acc)

        print('{} epoch, Loss: {:.6f}, Accuracy: {}'.format(epoch + 1, running_loss / (len(train)),running_acc))

        model.eval()
        eval_loss = 0
        eval_acc = 0
        for data,target in val_loader:
            data, target = Variable(data.cuda(async=True)),Variable(target.cuda(async=True))
            out = model(data)
            loss = criterion(out, target.float())
            eval_loss += loss.data[0] * target.size(0)

        print('Val Loss: {:.6f},Val acc: {}'.format(eval_loss / (len(val)),running_acc))
        val_losses.append(eval_loss / (len(val)))
        print()

In [96]:
fit(train_set, val_set, 10, 128)

Epoch 1
*****:
[1/10] Loss: 0.034080


/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


[1/10] Loss: 0.053671
[1/10] Loss: 0.038804
[1/10] Loss: 0.043166
[1/10] Loss: 0.046144
[1/10] Loss: 0.047518
[1/10] Loss: 0.044091
[1/10] Loss: 0.048826
[1/10] Loss: 0.045191
[1/10] Loss: 0.041261
Finish 1 epoch, Loss: 0.043902
VALIDATION Loss: 0.350931

Epoch 2
*****:


/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([51])) that is different to the input size (torch.Size([51, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([111])) that is different to the input size (torch.Size([111, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


[2/10] Loss: 0.040799
[2/10] Loss: 0.039230
[2/10] Loss: 0.038758
[2/10] Loss: 0.038372
[2/10] Loss: 0.043330
[2/10] Loss: 0.043854
[2/10] Loss: 0.041025
[2/10] Loss: 0.040208
[2/10] Loss: 0.038148
[2/10] Loss: 0.035363
Finish 2 epoch, Loss: 0.037626
VALIDATION Loss: 0.317635

Epoch 3
*****:
[3/10] Loss: 0.034411
[3/10] Loss: 0.039416
[3/10] Loss: 0.033880
[3/10] Loss: 0.041427
[3/10] Loss: 0.036900
[3/10] Loss: 0.037917
[3/10] Loss: 0.037353
[3/10] Loss: 0.035682
[3/10] Loss: 0.035862
[3/10] Loss: 0.032901
Finish 3 epoch, Loss: 0.035007
VALIDATION Loss: 0.296032

Epoch 4
*****:
[4/10] Loss: 0.026442
[4/10] Loss: 0.022982
[4/10] Loss: 0.024222
[4/10] Loss: 0.025618
[4/10] Loss: 0.023185
[4/10] Loss: 0.023085
[4/10] Loss: 0.022951
[4/10] Loss: 0.023864
[4/10] Loss: 0.024960
[4/10] Loss: 0.026406
Finish 4 epoch, Loss: 0.028096
VALIDATION Loss: 0.292506

Epoch 5
*****:
[5/10] Loss: 0.026339
[5/10] Loss: 0.018210
[5/10] Loss: 0.022622
[5/10] Loss: 0.022965
[5/10] Loss: 0.021072
[5/10] Loss